In [1]:
import os
os.environ['JAX_PLATFORMS'] = ''

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from os import listdir
from cv2 import imread
from cv2 import imshow
from cv2 import resize
from keras.applications import EfficientNetB0
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Flatten, Dropout
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint


In [3]:
base_path = "E:/UDA_LEARNING/UDA_DO AN TRI TUE NHAN TAO & UNG DUNG/BREAST CANCER/data/IDC_regular_ps50_idx5"
folder = listdir(base_path)
len(folder)
print(folder[:5])

['10253', '10254', '10255', '10256', '10257']


In [4]:
# how many image input?
total_images = 0
for n in range(len(folder)):
    patient_id = folder[n]
    for c in [0, 1]:
        patient_path = base_path + "/" + patient_id
        class_path = patient_path + "/" + str(c) + "/"
        subfiles = listdir(class_path)
        total_images += len(subfiles)

print(total_images)

277524


In [5]:
# split patient into frame: patient_id, path, target (id patient, path/to/patient and label)
data = pd.DataFrame(index=np.arange(0, total_images), columns=["patient_id", "path", "target"])

k = 0
for n in range(len(folder)):
    patient_id = folder[n]
    patient_path = base_path + "/" + patient_id 
    for c in [0,1]:
        class_path = patient_path + "/" + str(c) + "/"
        subfiles = listdir(class_path)
        for m in range(len(subfiles)):
            image_path = subfiles[m]
            data.loc[k, "path"] = class_path + image_path
            data.loc[k, "target"] = c
            data.loc[k, "patient_id"] = patient_id
            k += 1  

data.head()

,patient_id,path,target
0,10253,E:/UDA_LEARNING/UDA_DO AN TRI TUE NHAN TAO & U...,0
1,10253,E:/UDA_LEARNING/UDA_DO AN TRI TUE NHAN TAO & U...,0
2,10253,E:/UDA_LEARNING/UDA_DO AN TRI TUE NHAN TAO & U...,0
3,10253,E:/UDA_LEARNING/UDA_DO AN TRI TUE NHAN TAO & U...,0
4,10253,E:/UDA_LEARNING/UDA_DO AN TRI TUE NHAN TAO & U...,0


In [6]:
Pos_path = data.groupby("target").get_group(0)
Neg_path = data.groupby('target').get_group(1)

In [7]:
print(Neg_path[0:5])
print(len(Neg_path))

    patient_id                                               path target
479      10253  E:/UDA_LEARNING/UDA_DO AN TRI TUE NHAN TAO & U...      1
480      10253  E:/UDA_LEARNING/UDA_DO AN TRI TUE NHAN TAO & U...      1
481      10253  E:/UDA_LEARNING/UDA_DO AN TRI TUE NHAN TAO & U...      1
482      10253  E:/UDA_LEARNING/UDA_DO AN TRI TUE NHAN TAO & U...      1
483      10253  E:/UDA_LEARNING/UDA_DO AN TRI TUE NHAN TAO & U...      1
78786


In [8]:
lowerIndex = 0
upperIndex = 50000

Pos_train_path = list(Pos_path[lowerIndex: upperIndex]['path'])
Pos_train_target = list(Pos_path[lowerIndex: upperIndex]['target'])

print(len(Pos_train_path))
print(len(Pos_train_target))


50000
50000


In [9]:
Neg_train_path = list(Neg_path[lowerIndex: upperIndex]['path'])
Neg_train_target = list(Neg_path[lowerIndex: upperIndex]['target'])

print(len(Pos_train_path))
print(len(Neg_train_target))

50000
50000


In [10]:
x_train = []
y_train = []
x_test = []
y_test = []

WIDTH = 50
HEIGHT = 50

for path in Pos_train_path:
    origImageSize = imread(path)
    resizedImage = resize(origImageSize, (WIDTH, HEIGHT), interpolation=cv2.INTER_CUBIC)
    x_train.append(resizedImage)

for path in Neg_train_path:
    origImageSize = imread(path)
    resizedImage = resize(origImageSize, (WIDTH, HEIGHT), interpolation=cv2.INTER_CUBIC)
    x_train.append(resizedImage)

In [11]:
print(len(x_train))
y_train = Pos_train_target + Neg_train_target
print(len(y_train))

100000
100000


In [12]:
combined = list(zip(x_train, y_train))
np.random.shuffle(combined)
x_train, y_train = zip(*combined)

In [13]:
x_train = np.array(x_train)/255.0
y_train = np.array(y_train)

In [14]:
def create_EfficienNet(inputShape=(50, 50, 3), numClasses=1):
    baseModel = EfficientNetB0(weights=None, include_top=False, input_shape=inputShape)
    for layer in baseModel.layers:
        layer.trainable = False
    x = GlobalAveragePooling2D()(baseModel.output)
    x = Dense(256, activation='relu')(x)
    predictions = Dense(numClasses, activation='sigmoid')(x)
    model = Model(inputs=baseModel.input, outputs=predictions)    
    return model

In [15]:
model = create_EfficienNet(inputShape=(50, 50, 3), numClasses=1)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 50, 50, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 50, 50, 3) │          0 │ input_layer[0][0] │
│ (Rescaling)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 50, 50, 3) │          7 │ rescaling[0][0]   │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 51, 51, 3) │          0 │ normalization[0]… │
│ (ZeroPadding2D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 25, 25,    │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 25, 25,    │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 25, 25,    │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 25, 25,    │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 25, 25,    │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 25, 25,    │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 25, 25,    │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 25, 25,    │        512 │ block1a_se_excit… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_bn  │ (None, 25, 25,    │         64 │ block1a_project_

 Total params: 4,377,764 (16.70 MB)

 Trainable params: 328,193 (1.25 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [16]:
callBack = ModelCheckpoint(
    filepath = "BreastCancerModel_EfficienNet.keras",
    save_weights_only=False,
    monitor='accuracy',
    mode='max',
    save_best_only=True,
)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, batch_size=64, epochs=10, validation_split=0.2, callbacks=[callBack])

Epoch 1/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 2834s 2s/step - accuracy: 0.5033 - loss: 0.6932 - val_accuracy: 0.4997 - val_loss: 0.6932
Epoch 2/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 4615s 4s/step - accuracy: 0.5017 - loss: 0.6931 - val_accuracy: 0.4997 - val_loss: 0.6932
Epoch 3/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 2992s 2s/step - accuracy: 0.5007 - loss: 0.6932 - val_accuracy: 0.4997 - val_loss: 0.6932
Epoch 4/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 2113s 2s/step - accuracy: 0.5009 - loss: 0.6932 - val_accuracy: 0.5003 - val_loss: 0.6931
Epoch 5/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 2144s 2s/step - accuracy: 0.4963 - loss: 0.6932 - val_accuracy: 0.5003 - val_loss: 0.6931
Epoch 6/10
 975/1250 ━━━━━━━━━━━━━━━━━━━━ 6:19 1s/step - accuracy: 0.4973 - loss: 0.6932

KeyboardInterrupt: 